<a href="https://colab.research.google.com/github/celefigs/collaborative_project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt

import nltk
import re

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

#from wordcloud import WordCloud

# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from IPython.display import display


# Load data

In [33]:
file_path = '../data/final_data.csv'
df = pd.read_csv(file_path, encoding='utf-8')

# Cleaning columns ans filtering "Birth Control"
filtered_df = df[df['condition'] == 'Birth Control'].drop(
    columns=['Unnamed: 0', 'condition', 'date', 'usefulCount', 'matched_generic_name', 'matched_type']
)

filtered_df.head()

,provided_drug_name,review,rating
0,Lybrel,"""I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge.\r\nThe positive side is that I didn&#039;t have any other side effects. The idea of being period free was so tempting... Alas.""",5
1,Ortho Evra,"""This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch""",8
3,Ethinyl estradiol / levonorgestrel,"""I had been on the pill for many years. When my doctor changed my RX to chateal, it was as effective. It really did help me by completely clearing my acne, this takes about 6 months though. I did not gain extra weight, or develop any emotional health issues. I stopped taking it bc I started using a more natural method of birth control, but started to take it bc I hate that my acne came back at age 28. I really hope symptoms like depression, or weight gain do not begin to affect me as I am older now. I&#039;m also naturally moody, so this may worsen things. I was in a negative mental rut today. Also I hope this doesn&#039;t push me over the edge, as I believe I am depressed. Hopefully it&#039;ll be just like when I was younger.""",8
4,Nexplanon,"""Started Nexplanon 2 months ago because I have a minimal amount of contraception&#039;s I can take due to my inability to take the hormone that is used in most birth controls. I&#039;m trying to give it time because it is one of my only options right now. But honestly if I had options I&#039;d get it removed.\r\nI&#039;ve never had acne problems in my life, and immediately broke out after getting it implanted. Sex drive is completely gone, and I used to have sex with my boyfriend a few days a week, now its completely forced and not even fun for me anymore. I mean I&#039;m on birth control because I like having sex but don&#039;t want to get pregnant, why take a birth control that takes away sex? Very unhappy and hope that I get it back with time or I&#039;m getting it removed.""",3
5,Etonogestrel,"""Nexplanon does its job. I can have worry free sex. The only thing is that my periods are sometimes light and sometimes heavy. Sometimes they go away and sometimes they show up unexpected. I also feel somewhat depressed. Not sure if its Nexplanon or not. I&#039;ve had Nexplanont for about 2 months now, but despite the side effects its the most effective birth control I&#039;ve ever used and I do not plan on taking it out.""",9


In [35]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28788 entries, 0 to 31250
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   provided_drug_name  28788 non-null  object
 1   review              28788 non-null  object
 2   rating              28788 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 899.6+ KB


In [37]:
filtered_df.rename(columns= {"provided_drug_name":"drugname"}, inplace= True)

In [39]:
#label  pos and neg

def label_rating(rating):
    if rating <= 5:
        return 'Negative'
    else:
        return 'Positive'

In [41]:
filtered_df['target'] = filtered_df['rating'].apply(label_rating)
filtered_df.head()

,drugname,review,rating,target
0,Lybrel,"""I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge.\r\nThe positive side is that I didn&#039;t have any other side effects. The idea of being period free was so tempting... Alas.""",5,Negative
1,Ortho Evra,"""This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch""",8,Positive
3,Ethinyl estradiol / levonorgestrel,"""I had been on the pill for many years. When my doctor changed my RX to chateal, it was as effective. It really did help me by completely clearing my acne, this takes about 6 months though. I did not gain extra weight, or develop any emotional health issues. I stopped taking it bc I started using a more natural method of birth control, but started to take it bc I hate that my acne came back at age 28. I really hope symptoms like depression, or weight gain do not begin to affect me as I am older now. I&#039;m also naturally moody, so this may worsen things. I was in a negative mental rut today. Also I hope this doesn&#039;t push me over the edge, as I believe I am depressed. Hopefully it&#039;ll be just like when I was younger.""",8,Positive
4,Nexplanon,"""Started Nexplanon 2 months ago because I have a minimal amount of contraception&#039;s I can take due to my inability to take the hormone that is used in most birth controls. I&#039;m trying to give it time because it is one of my only options right now. But honestly if I had options I&#039;d get it removed.\r\nI&#039;ve never had acne problems in my life, and immediately broke out after getting it implanted. Sex drive is completely gone, and I used to have sex with my boyfriend a few days a week, now its completely forced and not even fun for me anymore. I mean I&#039;m on birth control because I like having sex but don&#039;t want to get pregnant, why take a birth control that takes away sex? Very unhappy and hope that I get it back with time or I&#039;m getting it removed.""",3,Negative
5,Etonogestrel,"""Nexplanon does its job. I can have worry free sex. The only thing is that my periods are sometimes light and sometimes heavy. Sometimes they go away and sometimes they show up unexpected. I also feel somewhat depressed. Not sure if its Nexplanon or not. I&#039;ve had Nexplanont for about 2 months now, but despite the side effects its the most effective birth control I&#039;ve ever used and I do not plan on taking it out.""",9,Positive


In [43]:
filtered_df.isnull().sum()

drugname    0
review      0
rating      0
target      0
dtype: int64

In [45]:
pd.set_option('display.max_colwidth', None)

In [47]:
#Check out target distribution

target_distr = filtered_df["target"].value_counts()
target_distr*100/len(filtered_df)

target
Positive    57.444074
Negative    42.555926
Name: count, dtype: float64

In [49]:
# 57% of the ratings are "Positive" and  43% are "Negative". A bit umbalanced dataset

In [61]:
filtered_df.review.iloc[1]

'"This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch"'

# Split the data

In [63]:
# X = independent variable = review,  y = target variable (dependent) = rating

X = filtered_df['review']
y= filtered_df['target']

# why we are doing these? video: 1:20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
print("DataFrame dimensions:", filtered_df.shape)
print(f"Train dimensions: {X_train.shape, y_train.shape}")  
print(f"Test dimensions: {X_test.shape, y_test.shape}")  

DataFrame dimensions: (28788, 4)
Train dimensions: ((23030,), (23030,))
Test dimensions: ((5758,), (5758,))


In [69]:
# Check the distribution of classes in the training and testing sets

print(y_train.value_counts())
print(y_test.value_counts())

# Both has the same proportion, 74% --> stratified

target
Positive    13232
Negative     9798
Name: count, dtype: int64
target
Positive    3305
Negative    2453
Name: count, dtype: int64


In [71]:
X_train.head()

26256                                                     "I switched to this pill after using Sprintec for years and had instant weight gain. I gained 20 pounds in the first three months. No noticeable changes in mood, sex drive, or acne. My skin stayed very clear. I also stopped having a period completely and although it was wonderful I could not get used to it. The pregnancy panics along with the weight gain forced me to stop taking these pills."
22909                                         "I&#039;ve had it for 2 weeks so far no side effects yay!!! I don&#039;t get why people keep it in for years if it has crazy side effects like weight gain, suicide etc? The moment I gain 5 lbs I&#039;m getting it out. I have a very very regular workout and diet schd. The only pills that worked for me we&#039;re Yasmin and mini pill but I kept forgetting to take it after my sons birth hence the implanon."
17874                                                                                       

# Start with the pre-processing

In [87]:
# Seeing what reviews looks like
text = X_train[1]
print(text)

"This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch"


In [78]:
# Def RegEx

REPLACE_NO_SPACE = re.compile(r"[.;:!#\'?,\"()\[\]&]|\d+")  # delete punctuation and numbers
REPLACE_WITH_SPACE = re.compile(r"(<br\s*/><br\s*/>)|(\-)|(\/)") 

# We want to remove everything that is not a proper character. RegEx for non-alphanumerical characters.

In [80]:
#Preprocess function for "review" column

stop_words = set(stopwords.words('english'))

def preprocess_review(reviews):

    # Remove special characters
    reviews = re.sub(REPLACE_NO_SPACE, "", reviews)  #aqui tambien podemos usar RegexpTokenizer()
    reviews = re.sub(REPLACE_WITH_SPACE, " ", reviews)

    tokens = word_tokenize(reviews)

    # Lowercase and lemmatize
    lemmatizer = WordNetLemmatizer()

    lemmas = [lemmatizer.lemmatize(token.lower(), pos='v') for token in tokens if token.lower() not in stop_words]
    lemmas = [lemmatizer.lemmatize(token.lower(), pos='n') for token in lemmas if token.lower() not in stop_words]


    return lemmas

Sanity check ____________________________________________________________________________________________________________________________________________________________________________

In [91]:
start = text.find("of birth control. I&#039;m glad I went with")
end = start + len("of birth control. I&#039;m glad I went with")
print(f"Start position: {start}")
print(f"End position: {end}")

Start position: 38
End position: 81


In [95]:
text = filtered_df['review'][1][38:81]
print(text)

of birth control. I&#039;m glad I went with


In [97]:
# sanity check - DELETE
preprocess_review(text)

['birth', 'control', 'im', 'glad', 'go']

____________________________________________________________________________________________________________________________________________________________________________

In [83]:
#What is my weighting scheme? in this case, 

vectoriser = TfidfVectorizer(analyzer=preprocess_review)

X_train_tfidf = vectoriser.fit_transform(X_train)

In [84]:
# DataFrame dimensions: (31251, 3)
# Train dimensions: ((23030,), (23030,))
# Test dimensions: ((5758,), (5758,))

X_train_tfidf.shape # (num_1, num_2) num_1: rows, documents, num_2: unigrams, tokens

(23030, 11644)

In [85]:
#represent test documents in the Vector Space we've learned with the train documents
X_test_tfidf = vectoriser.transform(X_test)

In [86]:
X_test_tfidf.shape 


(5758, 11644)

nlp done

# Train a Naive Bayes classifier

In [94]:
from sklearn.naive_bayes import MultinomialNB

# Fit the model using training data
# The text information now is structured and weighted according to TF-IDF

nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train) #y_train are my labels

MultinomialNB()

**X_train_tfidf** contiene las características (entrada)

**y_train** contiene las etiquetas (salida esperada)

In [92]:
y_train.head()

26256    Negative
22909    Positive
17874    Negative
3553     Negative
18426    Negative
Name: label, dtype: object

# How good is my model? = Model Performance

In [107]:
print('Accuracy:', accuracy_score(y_test, nb.predict(X_test_tfidf))) 

# **y_test** -> true labels 
# **nb.predict(X_test_tfidf)** -> The labels predicted by the model ()).

# **Accuracy** --> 
# - It returns a value between 0 and 1 
# - Represents the proportion of correct predictions relative to the total predictions

Accuracy: 0.794373046196596


In [106]:
average_method = 'weighted'  # Should I use 'macro' or 'micro'?
# Choose micro when you want a global view of your model’s performance.
# Choose macro when you want to treat all classes equally, irrespective of their sizes.
# Choose weighted when dealing with imbalanced classes, as it takes class sizes into account.

print(f'Accuracy: {accuracy_score(y_test, nb.predict(X_test_tfidf)):.2f}')  #REMOVE?
print(f'F1 Score: {f1_score(y_test, nb.predict(X_test_tfidf), average=average_method):.2f}')
print(f'Precision: {precision_score(y_test, nb.predict(X_test_tfidf), average=average_method):.2f}')
print(f'Recall: {recall_score(y_test, nb.predict(X_test_tfidf), average=average_method):.2f}')

# print(f'ROC AUC: {roc_auc_score(y_test, nb.predict_proba(X_test_tfidf)):.2f}')


Accuracy: 0.79
F1 Score: 0.79
Precision: 0.81
Recall: 0.79


**Notes**:
average='weighted':
 This parameter is used to compute the metrics for multi-class classification. It accounts for the proportion of each class in the dataset. can change it based on  specific needs (e.g., average='macro' or average='micro').

Given that whe have a binary classification problem with a slight class imbalance (57% positive and 43% negative reviews), we select the next evaluation metrics for our model:

**Precision**: is the proportion of true positive predictions among all positive predictions.

In our context, precision will tell us how many of the reviews predicted as positive are actually positive. High precision means fewer false positives, which is important if you want to ensure that positive reviews are indeed seen as such.

**Recall** (Sensitivity): proportion of true positive predictions among all actual positives. TP/P

Will show how well our model identifies actual positive reviews. In a review context, high recall means that most positive reviews are correctly identified.

**F1 Score**  (Relates precision and recall)

The F1 score is particularly useful for balancing precision and recall, especially when have a slight imbalance. It gives a single score to evaluate your model’s performance, which is beneficial when want to optimize both false positives and false negatives.

As an additional metric: **ROC-AUC**  Measures the model’s ability to distinguish between the positive and negative classes across all thresholds.
Importance:  provides insights into how well your model performs across different thresholds, giving a broader view of its performance regardless of class imbalance.

Why do not use accuracy --< While accuracy can provide a general sense of how well the model performs, it's not the best metric for imbalanced datasets. Since we have a slight imbalance, it might give a misleading impression of model performance, especially if the model predicts the majority class more often.
>

# Wrap Up

In [23]:
vectoriser = TfidfVectorizer(analyzer=preprocess_review)
rf= RandomForestClassifier()

pipe = Pipeline([('vectoriser', vectoriser),
                 ('classifier', rf)])

pipe.fit(X_train, y_train)

NameError: name 'X_train' is not defined

### Notes
1. Estructura del Pipeline: ¿use**GridSearchCV** ?
2. **Construir el Modelo (Supervised )**:
   - **Modelo Base (Baseline Model)**: **Logistic Regression** or **Naive Bayes** como referencia inicial?
   - **SVM (Support Vector Machines)** -Natalia
   - **Random Forest**
   - **Deep Learning (LSTM)**: red neuronal?